# Kernel Tuner Simulation Mode Test 3

This notebook is intended to examine the simulation mode against the normal mode using a fixed seed. 


In [ ]:
from pathlib import Path
import json
import re
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from warnings import warn
from typing import Tuple

cachedirectory = Path("kernels/outputdata/simulation_mode")
savedirectory = Path(Path.cwd(), "figures", "simulation_mode")
mpl.rcParams["savefig.directory"] = savedirectory

In [ ]:
# arguments
kernelname = "gemm_cltune_opencl"
expected_keys_env = ['execution_time', 'overhead_time', 'total_benchmark_time', 'total_verification_time', 'total_compile_time', 'total_strategy_time', 'total_framework_time']
expected_keys_cache = ['time', 'times', 'GFLOP/s', 'temperature', 'compile_time', 'benchmark_time', 'verification_time', 'framework_time', 'strategy_time']

algorithm_displaynames = {
    'brute': "bruteforce",
    'random': "random",
    'ordered': "ordered_greedy_mls",
}
column_displaynames = {
    'compile_time': "Compile time",
    'benchmark_time': "Benchmark time",
    'verification_time': "Verification time",
    'strategy_time': "Strategy time",
    'framework_time': "Framework time",
    'total_time': "Total time"
}

plot_height = 6
plot_aspect = 2.0
plot_dpi = 250
sns.set_theme(rc={'figure.figsize':(plot_height*plot_aspect,plot_height)}, font_scale=1.2)

In [ ]:
def get_dataframes(target_algorithm=None, fixed_fevals=False) -> Tuple[pd.DataFrame, pd.DataFrame]:

    # collect the relevant cachefiles
    cachefiles : list[Path] = []
    for file in cachedirectory.iterdir():
        if file.is_file() and file.name.startswith(kernelname) and file.name.endswith('_output.json'):
            cachefiles.append(file)

    # create a list of dictionaries of the data
    cache_dicts = list()
    env_dicts = list()
    for filepath in cachefiles:
        # load the data
        envfilepath = Path(str(filepath).replace('_output', '_env'))
        assert envfilepath.exists()
        try:
            cachefile = json.load(filepath.open())
            envfile = json.load(envfilepath.open())
        except json.JSONDecodeError:
            warn(f"Could not load cachefile {filepath.name}")
            continue
        assert isinstance(cachefile, list)
        assert isinstance(envfile, dict)

        # combine the data into a dictionary
        device_name = envfile['device_name']
        is_simulated = "_mode=simulated" in filepath.name
        mode = 'simulated' if is_simulated else 'real'
        algorithm = algorithm_displaynames[re.search(r"_alg=([a-zA-Z]+)_", filepath.name).group(1).strip()]
        if target_algorithm is not None and not algorithm.startswith(target_algorithm):
            continue
        if fixed_fevals != ('_fixedfevals' in filepath.name):
            continue
        run_number = int(re.search(r"_#(\d+)_", filepath.name).group(1))
        # for each of the tuned configurations, create a dictionary with the expected items
        for value in cachefile:
            assert isinstance(value, dict)
            cache = {}
            cache['device_name'] = device_name
            cache['algorithm'] = algorithm
            cache['simulated'] = is_simulated
            cache['mode'] = mode
            cache['run_number'] = run_number
            for key in expected_keys_cache:
                cache[key] = value[key]
            cache['total_time'] = value['strategy_time'] + value['compile_time'] + value['benchmark_time'] + value['verification_time'] + value['framework_time']
            cache_dicts.append(cache)
        # create a global dictionary per tuning session
        env = {}
        env['device_name'] = device_name
        env['algorithm'] = algorithm
        env['simulated'] = is_simulated
        env['mode'] = mode
        env['number_of_configs'] = len(cachefile)
        env['run_number'] = run_number
        for key in expected_keys_env:
            env[key] = envfile[key]
        env_dicts.append(env)

    # create a dataframe out of the dictionaries
    df_cache = pd.DataFrame(cache_dicts)
    df_env = pd.DataFrame(env_dicts)
    return df_cache, df_env

In [ ]:
df_cache, df_env = get_dataframes(target_algorithm='ordered')
df_cache